In [1]:
import os
import random

from ultralytics import YOLO
import cv2
import logging
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
from PIL import Image
import zipfile
import numpy as np
from skimage import io
from skimage.restoration import denoise_tv_chambolle
import math

In [2]:


# def unzip_files(zip_file_path):
#     with zipfile.ZipFile(zip_file_path, 'r') as zip_ref:
#         zip_ref.extractall()


# zip_file_path = 'archive.zip'


# unzip_files(zip_file_path)

In [3]:
def load_labels(image_file, train_labels):
    label_file = os.path.splitext(image_file)[0] + ".txt"
    label_path = os.path.join(train_labels, label_file)
    
    with open(label_path, "r") as f:
        labels = f.read().strip().split("\n")
    
    return labels

def plot_object_detections(ax, image, labels):
    for label in labels:
        if len(label.split()) != 5:
            continue
        class_id, x_center, y_center, width, height = map(float, label.split())
        x_min = int((x_center - width/2) * image.shape[1])
        y_min = int((y_center - height/2) * image.shape[0])
        x_max = int((x_center + width/2) * image.shape[1])
        y_max = int((y_center + height/2) * image.shape[0])
        cv2.rectangle(image, (x_min, y_min), (x_max, y_max), (0, 255, 0), 3)

    ax.imshow(cv2.cvtColor(image, cv2.COLOR_BGR2RGB))
    ax.axis('off')




In [9]:
# random_images = random.sample(image_files, 16)
model = YOLO("yolov8x.pt") 



results = model.val(data='data_1.yaml')
   

Ultralytics YOLOv8.2.2  Python-3.9.13 torch-2.0.0 CUDA:0 (NVIDIA GeForce RTX 3060, 12288MiB)
YOLOv8x summary (fused): 268 layers, 68200608 parameters, 0 gradients, 257.8 GFLOPs


100%|██████████| 755k/755k [00:00<00:00, 19.1MB/s]
val: Scanning C:\Users\Sway\ece50024\datasets\valid\labels... 300 images, 0 backgrounds, 0 corrupt: 100%|██████████| 300/300 [00:00<00:00, 872.91it/s]

val: WARNING  C:\Users\Sway\ece50024\datasets\valid\images\Pias--359-_PNG.rf.43bcf36efe5cf8c37552d2c45fffea60.jpg: 9 duplicate labels removed
val: WARNING  C:\Users\Sway\ece50024\datasets\valid\images\Pias--360-_PNG.rf.8405b0e44009a9300e0a1100ccf7d5b3.jpg: 7 duplicate labels removed
val: New cache created: C:\Users\Sway\ece50024\datasets\valid\labels.cache



                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 19/19 [00:09<00:00,  1.96it/s]


                   all        300       2568    0.00709     0.0629    0.00419    0.00327
            motorcycle        300         32    0.00516      0.906    0.00683    0.00293
              airplane        300        425     0.0833    0.00235     0.0463     0.0417
                   bus        300        842     0.0145     0.0451    0.00757    0.00486
                  boat        300          2          0          0          0          0
         traffic light        300        110          0          0          0          0
          fire hydrant        300        335          0          0          0          0
             stop sign        300        142          0          0          0          0
         parking meter        300          1          0          0          0          0
                 bench        300        192     0.0105     0.0521    0.00633    0.00276
                  bird        300          1          0          0          0          0
                   ca

In [4]:

# Functions for image -> image np array

def image_to_normalized_array(image):
    """
    Converts a PIL Image object to a normalized (0.0 to 1.0) float32 image array.

    Parameters:
        image (PIL.Image): The input PIL Image object.

    Returns:
        numpy.ndarray: The normalized image array with pixel values between 0.0 and 1.0.
    """
    # Convert the PIL Image to a NumPy array
    img_array = np.asarray(image)

    # Convert the array's data type to float32 for normalization
    img_array = img_array.astype(np.float32)

    # Normalize pixel values to be between 0.0 and 1.0
    img_array /= 255.0

    return img_array

In [31]:
# Best version

# Plug-and-Play ADMM denoising


def update_x(x_prev, v_prev, u_prev, rho, f):
    x = (f + rho * (v_prev - u_prev)) / (1 + rho)
    return x

def denoiser(image, sigma):
    return denoise_tv_chambolle(image, weight=sigma)

def update_v(x, u_prev, lambda_val, rho):
    sigma = np.sqrt(lambda_val / rho)
    return denoiser(x + u_prev, sigma)

def update_u(u_prev, x, v):
    return u_prev + (x - v)

def calc_delta(x, x_prev, v, v_prev, u, u_prev):
    delta = np.sqrt(np.mean((x - x_prev)**2) + np.mean((v - v_prev)**2) + np.mean((u - u_prev)**2))
    return delta

def plug_and_play_admm(f, lambda_val, rho_0, eta, gamma, tol, max_iterations):
    x = f.copy()
    v = f.copy()
    u = np.zeros_like(f)

    rho = rho_0
    delta = np.inf
    k = 0

    while delta >= tol and k < max_iterations:
        x_prev = x.copy()
        v_prev = v.copy()
        u_prev = u.copy()

        x = update_x(x_prev, v_prev, u_prev, rho, f)
        v = update_v(x, u_prev, lambda_val, rho)
        u = update_u(u_prev, x, v)

        delta_prev = delta
        delta = calc_delta(x, x_prev, v, v_prev, u, u_prev)

        if delta >= eta * delta_prev:
            rho *= gamma

        k += 1

    return x

def noise_gray(image, noise_level):
  noise = np.random.normal(scale=noise_level, size=image.shape)
  noisy_image = image + noise
  noisy_image = np.clip(noisy_image, 0, 255).astype(np.uint8)
  return noisy_image

# image = cv2.imread(dir_path + 'dog_on_grass.jpg')
# image_gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
# image_array = image_to_normalized_array(image_gray)

# noisy_image = noise_gray(image_gray, 30)
# noisy_array = image_to_normalized_array(noisy_image)

# Example usage:
# f: Blurred image
# lambda_val: Regularization parameter
# rho_0: Initial value of rho
# eta: Convergence criteria parameter
# gamma: Parameter for adjusting rho
# tol: Convergence tolerance
# max_iterations: Maximum number of iterations
# deblurred_image = plug_and_play_admm(f, lambda_val, rho_0, eta, gamma, tol, max_iterations)
# denoised_array = plug_and_play_admm(noisy_array, lambda_val=0.001, rho_0=0.00001, eta=0.5, gamma=1.4, tol=0.001, max_iterations=100)


# # Calculate PSNR and SSIM for noisy image
# psnr_noisy, ssim_noisy = compare_images(img_array, noisy_array)
# psnr_denoised, ssim_denoised = compare_images(img_array, denoised_array)
# print(f"Noisy Image PSNR: {psnr_noisy:.2f}, SSIM: {ssim_noisy:.4f}")
# print(f"Denoised Image PSNR: {psnr_denoised:.2f}, SSIM: {ssim_denoised:.4f}")

# # Display the deblurred image
# plt.figure(figsize=(16, 8))

# plt.subplot(1, 3, 1)
# plt.imshow(image_array, cmap='gray')
# plt.title('Original Image')

# plt.subplot(1, 3, 2)
# plt.imshow(noisy_array, cmap='gray')
# plt.title('Blurred Image')

# plt.subplot(1, 3, 3)
# plt.imshow(denoised_array, cmap='gray')
# plt.title('Deblurred Image')

# plt.show()


SyntaxError: invalid syntax (1925093448.py, line 65)

In [22]:
def add_noise_and_save(dir_path):
    for filename in os.listdir(dir_path):
        if filename.endswith(".jpg") or filename.endswith(".png"):
            img_path = os.path.join(dir_path, filename)
            image = cv2.imread(img_path)
            image_gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
            noisy_image = noise_gray(image_gray, 30)
            noisy_img_path = os.path.join("gray_noise_images", '' + filename)
            cv2.imwrite(noisy_img_path, noisy_image)
add_noise_and_save('original/')

In [23]:
results = model.val(data='data_1.yaml')

Ultralytics YOLOv8.2.2  Python-3.9.13 torch-2.0.0 CUDA:0 (NVIDIA GeForce RTX 3060, 12288MiB)


val: Scanning C:\Users\Sway\ece50024\datasets\valid\labels... 300 images, 0 backgrounds, 0 corrupt: 100%|██████████| 300/300 [00:00<00:00, 956.02it/s]

val: WARNING  C:\Users\Sway\ece50024\datasets\valid\images\Pias--359-_PNG.rf.43bcf36efe5cf8c37552d2c45fffea60.jpg: 9 duplicate labels removed
val: WARNING  C:\Users\Sway\ece50024\datasets\valid\images\Pias--360-_PNG.rf.8405b0e44009a9300e0a1100ccf7d5b3.jpg: 7 duplicate labels removed
val: New cache created: C:\Users\Sway\ece50024\datasets\valid\labels.cache



                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 19/19 [00:09<00:00,  2.10it/s]


                   all        300       2568    0.00937     0.0354    0.00586    0.00283
            motorcycle        300         32     0.0048      0.375    0.00498    0.00215
              airplane        300        425     0.0392    0.00941     0.0202     0.0141
                   bus        300        842     0.0125     0.0428     0.0066    0.00379
                  boat        300          2          0          0          0          0
         traffic light        300        110          0          0          0          0
          fire hydrant        300        335     0.0706     0.0896     0.0427     0.0168
             stop sign        300        142          0          0          0          0
         parking meter        300          1          0          0          0          0
                 bench        300        192      0.014     0.0417     0.0145    0.00606
                  bird        300          1          0          0          0          0
                   ca

In [33]:
import os
import cv2
import numpy as np

def denoise_and_save(input_dir_path, output_dir_path):
    for filename in os.listdir(input_dir_path):
        if filename.endswith(".jpg") or filename.endswith(".png"):
            
            img_path = os.path.join(input_dir_path, filename)
            image = cv2.imread(img_path)
            
            image_array = image_to_normalized_array(image)

            
            denoised_array = plug_and_play_admm(image_array, lambda_val=0.001, rho_0=0.00001, eta=0.5, gamma=1.4, tol=0.001, max_iterations=100)

           
            denoised_image = (denoised_array * 255).astype(np.uint8)

          
            denoised_img_path = os.path.join(output_dir_path, '' + filename)
            cv2.imwrite(denoised_img_path, denoised_image)


# denoise_and_save('gray_noise_images', 'gray_denoise_images')


In [6]:
#train yolo pretrained model with original datasets
odel = YOLO("yolov8x.pt") 
results = model.val(data='data_1.yaml')

Ultralytics YOLOv8.2.2  Python-3.9.13 torch-2.0.0 CUDA:0 (NVIDIA GeForce RTX 3060, 12288MiB)
YOLOv8x summary (fused): 268 layers, 68200608 parameters, 0 gradients, 257.8 GFLOPs


val: Scanning C:\Users\Sway\ece50024\datasets\valid\labels... 300 images, 0 backgrounds, 0 corrupt: 100%|██████████| 300/300 [00:00<00:00, 519.66it/s]

val: WARNING  C:\Users\Sway\ece50024\datasets\valid\images\Pias--359-_PNG.rf.43bcf36efe5cf8c37552d2c45fffea60.jpg: 9 duplicate labels removed
val: WARNING  C:\Users\Sway\ece50024\datasets\valid\images\Pias--360-_PNG.rf.8405b0e44009a9300e0a1100ccf7d5b3.jpg: 7 duplicate labels removed
val: New cache created: C:\Users\Sway\ece50024\datasets\valid\labels.cache



                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 19/19 [00:14<00:00,  1.32it/s]


                   all        300       2568    0.00706     0.0424    0.00448     0.0021
            motorcycle        300         32    0.00435      0.469    0.00458    0.00208
              airplane        300        425          0          0          0          0
                   bus        300        842     0.0274     0.0641     0.0147    0.00857
                  boat        300          2          0          0          0          0
         traffic light        300        110          0          0          0          0
          fire hydrant        300        335     0.0525     0.0627     0.0335     0.0145
             stop sign        300        142          0          0          0          0
         parking meter        300          1          0          0          0          0
                 bench        300        192     0.0116     0.0312    0.00981    0.00307
                  bird        300          1          0          0          0          0
                   ca

In [8]:

model.train(data = 'data_1.yaml',
            epochs = 30,
            imgsz = 360,
            seed = 42,
            batch = 8,
            workers = 4)

Ultralytics YOLOv8.2.2  Python-3.9.13 torch-2.0.0 CUDA:0 (NVIDIA GeForce RTX 3060, 12288MiB)
engine\trainer: task=detect, mode=train, model=yolov8x.pt, data=data_1.yaml, epochs=30, time=None, patience=100, batch=8, imgsz=360, save=True, save_period=-1, cache=False, device=None, workers=4, project=None, name=train, exist_ok=False, pretrained=True, optimizer=auto, verbose=True, seed=42, deterministic=True, single_cls=False, rect=False, cos_lr=False, close_mosaic=10, resume=False, amp=True, fraction=1.0, profile=False, freeze=None, multi_scale=False, overlap_mask=True, mask_ratio=4, dropout=0.0, val=True, split=val, save_json=False, save_hybrid=False, conf=None, iou=0.7, max_det=300, half=False, dnn=False, plots=True, source=None, vid_stride=1, stream_buffer=False, visualize=False, augment=False, agnostic_nms=False, classes=None, retina_masks=False, embed=None, show=False, save_frames=False, save_txt=False, save_conf=False, save_crop=False, show_labels=True, show_conf=True, show_boxes=Tru

100%|██████████| 6.23M/6.23M [00:00<00:00, 66.6MB/s]


AMP: checks passed 
WARNING  imgsz=[360] must be multiple of max stride 32, updating to [384]


train: Scanning C:\Users\Sway\ece50024\datasets\train\labels... 2624 images, 1 backgrounds, 0 corrupt: 100%|██████████| 2624/2624 [00:03<00:00, 827.24it/s]


train: New cache created: C:\Users\Sway\ece50024\datasets\train\labels.cache


...

                   all        300       2568      0.591      0.266      0.268       0.16



30 epochs completed in 1.216 hours.
Optimizer stripped from runs\detect\train\weights\last.pt, 136.7MB
Optimizer stripped from runs\detect\train\weights\best.pt, 136.7MB

Validating runs\detect\train\weights\best.pt...
Ultralytics YOLOv8.2.2  Python-3.9.13 torch-2.0.0 CUDA:0 (NVIDIA GeForce RTX 3060, 12288MiB)
Model summary (fused): 268 layers, 68143791 parameters, 0 gradients, 257.5 GFLOPs


                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 19/19 [00:06<00:00,  3.05it/s]


                   all        300       2568      0.619      0.243      0.268      0.172
               bicycle        300         32      0.339      0.125      0.137     0.0431
                   bus        300        425       0.72      0.315      0.443      0.276
                   car        300        842      0.725      0.576      0.643      0.415
               minibus        300          2      0.289        0.5      0.501      0.449
               minivan        300        110       0.37      0.382      0.319      0.221
             motorbike        300        335      0.632      0.288      0.345      0.109
                pickup        300        142       0.87     0.0942      0.202      0.133
             policecar        300          1          1          0          0          0
              rickshaw        300        192      0.657       0.31      0.368      0.216
               scooter        300          1          1          0          0          0
                   su

ultralytics.utils.metrics.DetMetrics object with attributes:

ap_class_index: array([ 3,  4,  5,  8,  9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20])
box: ultralytics.utils.metrics.Metric object
confusion_matrix: <ultralytics.utils.metrics.ConfusionMatrix object at 0x000002DE962779D0>
curves: ['Precision-Recall(B)', 'F1-Confidence(B)', 'Precision-Confidence(B)', 'Recall-Confidence(B)']
curves_results: [[array([          0,    0.001001,    0.002002,    0.003003,    0.004004,    0.005005,    0.006006,    0.007007,    0.008008,    0.009009,     0.01001,    0.011011,    0.012012,    0.013013,    0.014014,    0.015015,    0.016016,    0.017017,    0.018018,    0.019019,     0.02002,    0.021021,    0.022022,    0.023023,
          0.024024,    0.025025,    0.026026,    0.027027,    0.028028,    0.029029,     0.03003,    0.031031,    0.032032,    0.033033,    0.034034,    0.035035,    0.036036,    0.037037,    0.038038,    0.039039,     0.04004,    0.041041,    0.042042,    0.043043,    0.04

In [9]:
#oringinal images validation with trained model
results = model.val(data='data_1.yaml')

Ultralytics YOLOv8.2.2  Python-3.9.13 torch-2.0.0 CUDA:0 (NVIDIA GeForce RTX 3060, 12288MiB)
Model summary (fused): 268 layers, 68143791 parameters, 0 gradients, 257.5 GFLOPs


val: Scanning C:\Users\Sway\ece50024\datasets\valid\labels... 300 images, 0 backgrounds, 0 corrupt: 100%|██████████| 300/300 [00:00<00:00, 462.32it/s]

val: WARNING  C:\Users\Sway\ece50024\datasets\valid\images\Pias--359-_PNG.rf.43bcf36efe5cf8c37552d2c45fffea60.jpg: 9 duplicate labels removed
val: WARNING  C:\Users\Sway\ece50024\datasets\valid\images\Pias--360-_PNG.rf.8405b0e44009a9300e0a1100ccf7d5b3.jpg: 7 duplicate labels removed
val: New cache created: C:\Users\Sway\ece50024\datasets\valid\labels.cache



                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 38/38 [00:10<00:00,  3.68it/s]


                   all        300       2568      0.586      0.413      0.419      0.262
               bicycle        300         32      0.418      0.375      0.429      0.169
                   bus        300        425      0.698      0.586      0.657      0.431
                   car        300        842      0.739      0.729      0.766      0.515
               minibus        300          2      0.199      0.199      0.252      0.201
               minivan        300        110      0.357      0.526      0.397      0.284
             motorbike        300        335      0.502      0.546      0.481      0.167
                pickup        300        142      0.829      0.225      0.377      0.245
             policecar        300          1          1          0          0          0
              rickshaw        300        192      0.586      0.672      0.664      0.427
               scooter        300          1          1          0          0          0
                   su

In [10]:
#noise images validation with trained model
results = model.val(data='data_1.yaml')

Ultralytics YOLOv8.2.2  Python-3.9.13 torch-2.0.0 CUDA:0 (NVIDIA GeForce RTX 3060, 12288MiB)


val: Scanning C:\Users\Sway\ece50024\datasets\valid\labels... 300 images, 0 backgrounds, 0 corrupt: 100%|██████████| 300/300 [00:00<00:00, 712.90it/s]

val: WARNING  C:\Users\Sway\ece50024\datasets\valid\images\Pias--359-_PNG.rf.43bcf36efe5cf8c37552d2c45fffea60.jpg: 9 duplicate labels removed
val: WARNING  C:\Users\Sway\ece50024\datasets\valid\images\Pias--360-_PNG.rf.8405b0e44009a9300e0a1100ccf7d5b3.jpg: 7 duplicate labels removed
val: New cache created: C:\Users\Sway\ece50024\datasets\valid\labels.cache



                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 38/38 [00:08<00:00,  4.44it/s]


                   all        300       2568      0.367      0.107     0.0955     0.0543
               bicycle        300         32          0          0    0.00544    0.00151
                   bus        300        425      0.503     0.0689      0.137     0.0712
                   car        300        842      0.511      0.437      0.447      0.255
               minibus        300          2    0.00739     0.0111      0.172      0.152
               minivan        300        110       0.11      0.318      0.111     0.0632
             motorbike        300        335      0.395     0.0896      0.113     0.0313
                pickup        300        142          0          0     0.0265     0.0108
             policecar        300          1          1          0          0          0
              rickshaw        300        192      0.227     0.0938     0.0907     0.0475
               scooter        300          1          1          0          0          0
                   su

In [11]:
#denoise images validation with trained model
results = model.val(data='data_1.yaml')

Ultralytics YOLOv8.2.2  Python-3.9.13 torch-2.0.0 CUDA:0 (NVIDIA GeForce RTX 3060, 12288MiB)


val: Scanning C:\Users\Sway\ece50024\datasets\valid\labels... 300 images, 0 backgrounds, 0 corrupt: 100%|██████████| 300/300 [00:00<00:00, 914.67it/s]

val: WARNING  C:\Users\Sway\ece50024\datasets\valid\images\Pias--359-_PNG.rf.43bcf36efe5cf8c37552d2c45fffea60.jpg: 9 duplicate labels removed
val: WARNING  C:\Users\Sway\ece50024\datasets\valid\images\Pias--360-_PNG.rf.8405b0e44009a9300e0a1100ccf7d5b3.jpg: 7 duplicate labels removed
val: New cache created: C:\Users\Sway\ece50024\datasets\valid\labels.cache



                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 38/38 [00:09<00:00,  4.18it/s]


                   all        300       2568      0.616      0.246      0.268      0.172
               bicycle        300         32       0.33      0.125      0.135     0.0432
                   bus        300        425      0.709      0.316      0.442      0.278
                   car        300        842       0.72       0.58      0.644      0.415
               minibus        300          2      0.283        0.5      0.501      0.449
               minivan        300        110      0.358      0.382      0.319      0.223
             motorbike        300        335      0.634      0.295      0.343       0.11
                pickup        300        142      0.871     0.0949      0.202      0.133
             policecar        300          1          1          0          0          0
              rickshaw        300        192      0.661      0.314       0.37      0.218
               scooter        300          1          1          0          0          0
                   su